In [1]:
cd(@__DIR__) #makes the directory where this script is located the new working directory
using Pkg
Pkg.activate()
Pkg.instantiate()
using MixedModels
using RCall
using DataFrames
using CSV
using RData
using Statistics
using Dates

 Activating environment at `C:\Users\bjoer\.julia\environments\v1.4\Project.toml`



R version 3.6.0 (2019-04-26) -- "Planting of a Tree"
Copyright (C) 2019 The R Foundation for Statistical Computing
Platform: x86_64-w64-mingw32/x64 (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.



┌ Warning: RCall.jl: This version of Shiny is designed to work with 'htmlwidgets' >= 1.5.
│     Please upgrade via install.packages('htmlwidgets').
│ Registered S3 method overwritten by 'xts':
│   method     from
│   as.zoo.xts zoo 
│ [Previously saved workspace restored]
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113


In [2]:
R"""
require(dplyr, quietly = TRUE)   # for data wrangling
require(tidyverse, quietly = TRUE)   # for data wrangling
require(lme4)
require(lmerTest)
require(quickpsy)

SimulatePsychometricFunction_Staircase = function(ID, 
                                                  ConditionOfInterest, 
                                                  StandardValues, 
                                                  reps, 
                                                  PSE_Difference, 
                                                  JND_Difference, 
                                                  Multiplicator_PSE_Standard, 
                                                  Multiplicator_SD_Standard, 
                                                  SD_ResponseFunction, 
                                                  Mean_Variability_Between = 0.1, 
                                                  SD_Variability_Between = 0.1){
  Psychometric = expand.grid(ID=ID, ConditionOfInterest=ConditionOfInterest, StandardValues=StandardValues, reps = reps)
  
  Psychometric = Psychometric %>%
    group_by(ID) %>%#
    mutate(PSE_Factor_ID = rnorm(1,1,Mean_Variability_Between),
           SD_Factor_ID = rnorm(1,1,SD_Variability_Between))
  
  Psychometric = Psychometric %>%
    mutate(
      Mean_Standard = StandardValues+StandardValues*Multiplicator_PSE_Standard,
      SD_Standard = StandardValues*Multiplicator_SD_Standard,
      Mean = (Mean_Standard + (ConditionOfInterest==ConditionOfInterest[2])*StandardValues*PSE_Difference)*PSE_Factor_ID,
      SD = abs((SD_Standard + (ConditionOfInterest==ConditionOfInterest[2])*SD_Standard*JND_Difference)*SD_Factor_ID),
      staircase_factor = rcauchy(length(reps),1,SD_ResponseFunction), 
      Presented_TestStimulusStrength = Mean*staircase_factor,
      Difference = Presented_TestStimulusStrength - StandardValues,
      AnswerProbability = pnorm(Presented_TestStimulusStrength,Mean,SD),
      Answer = as.numeric(rbernoulli(length(AnswerProbability),AnswerProbability))
    )
  
  Psychometric = Psychometric %>%
    filter(abs(staircase_factor-1) < 0.75) %>%
    group_by(ID,ConditionOfInterest,StandardValues,Difference) %>%
    mutate(Yes = sum(Answer==1),
           Total = length(ConditionOfInterest))
  
  Psychometric
}
""";

┌ Warning: RCall.jl: Warning: package 'dplyr' was built under R version 3.6.3
│ 
│ Attaching package: 'dplyr'
│ 
│ The following objects are masked from 'package:stats':
│ 
│     filter, lag
│ 
│ The following objects are masked from 'package:base':
│ 
│     intersect, setdiff, setequal, union
│ 
└ @ RCall C:\Users\bjoer\.julia\packages\RCall\g7dhB\src\io.jl:113
┌ Warning: RCall.jl: -- Attaching packages --------------------------------------- tidyverse 1.2.1 --
│ v ggplot2 3.2.0     v readr   1.3.1
│ v tibble  3.0.0     v purrr   0.3.3
│ v tidyr   1.0.2     v stringr 1.4.0
│ v ggplot2 3.2.0     v forcats 0.4.0
│ Warning: package 'tibble' was built under R version 3.6.3
│ Warning: package 'tidyr' was built under R version 3.6.3
│ Warning: package 'purrr' was built under R version 3.6.3
│ -- Conflicts ------------------------------------------ tidyverse_conflicts() --
│ x .GlobalEnv::%||%() masks purrr::%||%()
│ x dplyr::filter()    masks stats::filter()
│ x dplyr::lag()       masks sta

In [3]:
function SimulateDataframe_Twolevel(n,
        ConditionOfInterest,
        StandardValues,
        reps,
        PSE_Difference,
        JND_Difference,
        Multiplicator_PSE_Standard,
        Multiplicator_SD_Standard,
        SD_ResponseFunction,
        Mean_Variability_Between,
        SD_Variability_Between)
    
    @rput n ConditionOfInterest StandardValues reps PSE_Difference JND_Difference Multiplicator_PSE_Standard Multiplicator_SD_Standard SD_ResponseFunction Mean_Variability_Between SD_Variability_Between

    R"""
    ID = paste0("s",1:n)
        Psychometric = SimulatePsychometricFunction_Staircase(ID,
            ConditionOfInterest,
            StandardValues,
            1:reps,
            PSE_Difference,
            JND_Difference,
            Multiplicator_PSE_Standard,
            Multiplicator_SD_Standard,
            SD_ResponseFunction,
            Mean_Variability_Between,
            SD_Variability_Between)
    
        (Parameters = quickpsy(Psychometric,Difference,Answer,
                               grouping = .(ID,ConditionOfInterest,StandardValues), 
                               bootstrap = "none")$par)
        Parameters2 = Parameters %>%
        filter(parn == "p1") %>%
        select(ID,ConditionOfInterest,Mean=par, StandardValues)
        Parameters2$SD = Parameters$par[Parameters$parn == "p2"]
        FittedPsychometricFunctions = Parameters2
    
        FittedPsychometricFunctions$StandardValues = as.character(FittedPsychometricFunctions$StandardValues)
        Psychometric$StandardValues = as.character(Psychometric$StandardValues)
    
        ANOVA_Mean = lm(Mean ~ as.factor(ConditionOfInterest)*StandardValues,FittedPsychometricFunctions)
        Pvalue_Mean_ANOVA = summary(ANOVA_Mean)$coefficients[14]
        ANOVA_SD = lm(SD ~ as.factor(ConditionOfInterest)*StandardValues,FittedPsychometricFunctions)
        Pvalue_SD_ANOVA = summary(ANOVA_SD)$coefficients[14]
    
    """
    @rget Psychometric FittedPsychometricFunctions Pvalue_Mean_ANOVA Pvalue_SD_ANOVA
    
    Psychometric[:StandardValuesAsFactor] = "placeholder"
    
    formula1 = @formula(Answer ~ Difference*ConditionOfInterest + (Difference + ConditionOfInterest |ID) + (Difference|StandardValues));
    GLMM = fit!(GeneralizedLinearMixedModel(formula1, Psychometric, Binomial()), fast=true)
    
    formulaNull = @formula(Answer ~ Difference + ConditionOfInterest + (Difference + ConditionOfInterest |ID) + (Difference|StandardValues));
    GLMMNull = fit!(GeneralizedLinearMixedModel(formulaNull, Psychometric, Binomial()), fast=true)
    Pvalue_LRT = MixedModels.likelihoodratiotest(GLMM,GLMMNull).pvalues[1]
    
    
    formula2 = @formula(Mean ~ ConditionOfInterest + (1|ID) + (1|StandardValues));
    TwoLevelMean = fit(MixedModel,formula2, FittedPsychometricFunctions)
    
    formula3 = @formula(SD ~ ConditionOfInterest + (1|ID) + (1|StandardValues));
    TwoLevelSD = fit(MixedModel,formula3, FittedPsychometricFunctions)

    [(coeftable(GLMM)).cols[4][3];
        Pvalue_LRT;
        (coeftable(TwoLevelMean)).cols[4][2];
        (coeftable(TwoLevelSD)).cols[4][2];
        Pvalue_Mean_ANOVA;
        Pvalue_SD_ANOVA]
end

SimulateDataframe_Twolevel (generic function with 1 method)

In [6]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
Range_reps = [40,70,100]
Range_PSE_Difference = [-0.05,-0.025,0,0.025,0.05]
Range_JND_Difference = [-0.1,-0.05,0,0.05,0.1]
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.2
SD_Variability_Between = 0.2
nIterations = 100
Range_Participants = [10,12,14,16,18,20]

TotalNumber = length(Range_reps)*length(Range_PSE_Difference)*length(Range_JND_Difference)*length(Range_Participants)
CurrentRunthrough = 0
rightnow = Dates.now()

for reps in Range_reps
    for PSE_Difference in Range_PSE_Difference
        for JND_Difference in Range_JND_Difference
            for n in Range_Participants
                
                TimeStartTrial = Dates.now()
                
                Pvalues_Accuracy = []
                Pvalues_Precision = []
                Pvalues_Accuracy_TwoLevel = []
                Pvalues_Precision_TwoLevel = []
                Pvalues_Accuracy_ANOVA = []
                Pvalues_Precision_ANOVA = []
                
                for j in 1:nIterations
                Pvalues = SimulateDataframe_Twolevel(n, 
                                      ConditionOfInterest, 
                                      StandardValues, 
                                      reps, 
                                      PSE_Difference, 
                                      JND_Difference, 
                                      Multiplicator_PSE_Standard, 
                                      Multiplicator_SD_Standard, 
                                      SD_ResponseFunction, 
                                      Mean_Variability_Between, 
                                      SD_Variability_Between)
                                        
                    Pvalues_Accuracy = [Pvalues_Accuracy;Pvalues[1]]
                    Pvalues_Precision = [Pvalues_Precision;Pvalues[2]]
                    Pvalues_Accuracy_TwoLevel = [Pvalues_Accuracy_TwoLevel;Pvalues[3]]
                    Pvalues_Precision_TwoLevel = [Pvalues_Precision_TwoLevel;Pvalues[4]]
                    Pvalues_Accuracy_ANOVA = [Pvalues_Accuracy_ANOVA;Pvalues[5]]
                    Pvalues_Precision_ANOVA = [Pvalues_Precision_ANOVA;Pvalues[6]]
                end
                
                CurrentRunthrough = CurrentRunthrough + 1

                if CurrentRunthrough == 1

                   global PowerfulDataframe = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],
                        StandardValue2=StandardValues[2], reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        power_Accuracy_Twolevel = mean(Pvalues_Accuracy_TwoLevel .< 0.05),  
                        power_Precision_Twolevel = mean(Pvalues_Precision_TwoLevel .< 0.05),
                        power_Accuracy_ANOVA = mean(Pvalues_Accuracy_ANOVA .< 0.05),  
                        power_Precision_ANOVA = mean(Pvalues_Precision_ANOVA .< 0.05),
                        Duration = ((Dates.now()) - TimeStartTrial))

                else
                    row = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],StandardValue2=StandardValues[2], 
                        reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        power_Accuracy_Twolevel = mean(Pvalues_Accuracy_TwoLevel .< 0.05),  
                        power_Precision_Twolevel = mean(Pvalues_Precision_TwoLevel .< 0.05),
                        power_Accuracy_ANOVA = mean(Pvalues_Accuracy_ANOVA .< 0.05),  
                        power_Precision_ANOVA = mean(Pvalues_Precision_ANOVA .< 0.05),
                        Duration=((Dates.now()) - TimeStartTrial))
                    
                    PowerfulDataframe = append!(PowerfulDataframe,row)
                end
                
                print("RUNTHROUGH ", CurrentRunthrough, " out of ", TotalNumber,": ", n, " ", reps, " ", PSE_Difference, " ", JND_Difference, " ", mean(Pvalues_Accuracy .< 0.05), " ", 
                    mean(Pvalues_Precision .< 0.05), " ", PowerfulDataframe[!,:Duration][CurrentRunthrough], " END. ")

            end
        end
        CSV.write(join([reps,"_Big_", PSE_Difference, "_", JND_Difference, ".csv"]),PowerfulDataframe)
    end
end

|===================================================== | 99% ~0 s remaining     HROUGH 2 out of 450: 12 40 -0.05 -0.1 1.0 0.11 403679 milliseconds END. RUNTHROUGH 3 out of 450: 14 40 -0.05 -0.1 1.0 0.16 468543 milliseconds END.                                                                                   RUNTHROUGH 4 out of 450: 16 40 -0.05 -0.1 1.0 0.12 588970 milliseconds END.                                                                                                                                                                                                                                     RUNTHROUGH 5 out of 450: 18 40 -0.05 -0.1 1.0 0.27 700381 milliseconds END.                                                                                                                                                                                                                                                                                                                       

Excessive output truncated after 524298 bytes.

In [5]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
Range_reps = [70]
Range_PSE_Difference = [-0.05,-0.025,0,0.025,0.05]
Range_JND_Difference = [-0.1,-0.05,0,0.05,0.1]
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 100
Range_Participants = [10,12,14,16,18,20]

TotalNumber = length(Range_reps)*length(Range_PSE_Difference)*length(Range_JND_Difference)*length(Range_Participants)
CurrentRunthrough = 0
rightnow = Dates.now()

for reps in Range_reps
    for PSE_Difference in Range_PSE_Difference
        for JND_Difference in Range_JND_Difference
            for n in Range_Participants
                
                TimeStartTrial = Dates.now()
                
                Pvalues_Accuracy = []
                Pvalues_Precision = []
                Pvalues_Accuracy_TwoLevel = []
                Pvalues_Precision_TwoLevel = []
                Pvalues_Accuracy_ANOVA = []
                Pvalues_Precision_ANOVA = []
                
                for j in 1:nIterations
                Pvalues = SimulateDataframe_Twolevel(n, 
                                      ConditionOfInterest, 
                                      StandardValues, 
                                      reps, 
                                      PSE_Difference, 
                                      JND_Difference, 
                                      Multiplicator_PSE_Standard, 
                                      Multiplicator_SD_Standard, 
                                      SD_ResponseFunction, 
                                      Mean_Variability_Between, 
                                      SD_Variability_Between)
                    
                    print(Pvalues[5])
                    print(Pvalues[6])
                    
                    Pvalues_Accuracy = [Pvalues_Accuracy;Pvalues[1]]
                    Pvalues_Precision = [Pvalues_Precision;Pvalues[2]]
                    Pvalues_Accuracy_TwoLevel = [Pvalues_Accuracy_TwoLevel;Pvalues[3]]
                    Pvalues_Precision_TwoLevel = [Pvalues_Precision_TwoLevel;Pvalues[4]]
                    Pvalues_Accuracy_ANOVA = [Pvalues_Accuracy_ANOVA;Pvalues[5]]
                    Pvalues_Precision_ANOVA = [Pvalues_Precision_ANOVA;Pvalues[6]]
                end
                
                CurrentRunthrough = CurrentRunthrough + 1

                if CurrentRunthrough == 1

                   global PowerfulDataframe = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],
                        StandardValue2=StandardValues[2], reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        power_Accuracy_Twolevel = mean(Pvalues_Accuracy_TwoLevel .< 0.05),  
                        power_Precision_Twolevel = mean(Pvalues_Precision_TwoLevel .< 0.05),
                        power_Accuracy_ANOVA = mean(Pvalues_Accuracy_ANOVA .< 0.05),  
                        power_Precision_ANOVA = mean(Pvalues_Precision_ANOVA .< 0.05),
                        Duration = ((Dates.now()) - TimeStartTrial))   

                else
                    row = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],StandardValue2=StandardValues[2], 
                        reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        power_Accuracy_Twolevel = mean(Pvalues_Accuracy_TwoLevel .< 0.05),  
                        power_Precision_Twolevel = mean(Pvalues_Precision_TwoLevel .< 0.05),
                        power_Accuracy_ANOVA = mean(Pvalues_Accuracy_ANOVA .< 0.05),  
                        power_Precision_ANOVA = mean(Pvalues_Precision_ANOVA .< 0.05),
                        Duration=((Dates.now()) - TimeStartTrial))
                    
                    PowerfulDataframe = append!(PowerfulDataframe,row)
                end
                
                print("RUNTHROUGH ", CurrentRunthrough, " out of ", TotalNumber,": ", n, " ", reps, " ", PSE_Difference, " ", JND_Difference, " ", mean(Pvalues_Accuracy .< 0.05), " ", 
                    mean(Pvalues_Precision .< 0.05), " ", PowerfulDataframe[!,:Duration][CurrentRunthrough], " END. ")

            end
            CSV.write(join([reps,"_Big_", PSE_Difference, "_", JND_Difference, ".csv"]),PowerfulDataframe)
        end
    end
end

SimulateDataframe (generic function with 1 method)

In [47]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
Range_reps = [30,40,50,60]
Range_PSE_Difference = 0.1
Range_JND_Difference = 0.2
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 25
Range_Participants = [10,12,14,16,18,20]

TotalNumber = length(Range_reps)*length(Range_PSE_Difference)*length(Range_JND_Difference)*length(Range_Participants)
CurrentRunthrough = 0
rightnow = Dates.now()

for reps in Range_reps
    for PSE_Difference in Range_PSE_Difference
        for JND_Difference in Range_JND_Difference
            for n in Range_Participants
                
                Pvalues_Accuracy = []
                Pvalues_Precision = []
                DurationsGLMM = []
                DurationsDataframe = []

                for j in 1:nIterations
                Pvalues = SimulateDataframe(n, 
                                      ConditionOfInterest, 
                                      StandardValues, 
                                      reps, 
                                      PSE_Difference, 
                                      JND_Difference, 
                                      Multiplicator_PSE_Standard, 
                                      Multiplicator_SD_Standard, 
                                      SD_ResponseFunction, 
                                      Mean_Variability_Between, 
                                      SD_Variability_Between)
                    Pvalues_Accuracy = [Pvalues_Accuracy;Pvalues[1]]
                    Pvalues_Precision = [Pvalues_Precision;Pvalues[2]]
                    DurationsGLMM = [DurationsGLMM;Dates.value(Pvalues[3])]
                    DurationsDataframe = [DurationsDataframe;Dates.value(Pvalues[4])]
                end
                
                CurrentRunthrough = CurrentRunthrough + 1

                if CurrentRunthrough == 1

                   global PowerfulDataframe = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],
                        StandardValue2=StandardValues[2], reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        Mean_DurationGLMM = mean(DurationsGLMM),
                        SD_DurationGLMM = std(DurationsGLMM),
                        Mean_DurationDataframe = mean(DurationsDataframe),
                        SD_DurationDataframe = std(DurationsDataframe),
                        nIterations = nIterations)

                else
                    row = DataFrame(n=n, 
                        ConditionsOfInterest=length(ConditionOfInterest), 
                        StandardValue1=StandardValues[1],StandardValue2=StandardValues[2], 
                        reps=reps, 
                        PSE_Difference=PSE_Difference, 
                        JND_Difference=JND_Difference, 
                        Multiplicator_PSE_Standard=Multiplicator_PSE_Standard, 
                        Multiplicator_SD_Standard=Multiplicator_SD_Standard, 
                        SD_ResponseFunction=SD_ResponseFunction, 
                        Mean_Variability_Between=Mean_Variability_Between, 
                        SD_Variability_Between=SD_Variability_Between, 
                        power_Accuracy = mean(Pvalues_Accuracy .< 0.05),  
                        power_Precision = mean(Pvalues_Precision .< 0.05),
                        Mean_DurationGLMM = mean(DurationsGLMM),
                        SD_DurationGLMM = std(DurationsGLMM),
                        Mean_DurationDataframe = mean(DurationsDataframe),
                        SD_DurationDataframe = std(DurationsDataframe),
                        nIterations = nIterations)
                    
                    PowerfulDataframe = append!(PowerfulDataframe,row)
                end

                print("RUNTHROUGH ", CurrentRunthrough, " out of ", TotalNumber,": ", n, " ", reps, " ", 
                    PowerfulDataframe[!,:Mean_DurationGLMM][CurrentRunthrough], " END. ")

            end
        end
    end
end


5RUNTHROUGH 1 out of 24: 10 30 492.08 END. 5RUNTHROUGH 2 out of 24: 12 30 657.44 END. 5RUNTHROUGH 3 out of 24: 14 30 716.48 END. 5RUNTHROUGH 4 out of 24: 16 30 869.4 END. 5RUNTHROUGH 5 out of 24: 18 30 903.76 END. 5RUNTHROUGH 6 out of 24: 20 30 1113.72 END. 5RUNTHROUGH 7 out of 24: 10 40 725.96 END. 5RUNTHROUGH 8 out of 24: 12 40 826.36 END. 5RUNTHROUGH 9 out of 24: 14 40 1047.92 END. 5RUNTHROUGH 10 out of 24: 16 40 1806.08 END. 5RUNTHROUGH 11 out of 24: 18 40 1933.44 END. 5RUNTHROUGH 12 out of 24: 20 40 2041.36 END. 5RUNTHROUGH 13 out of 24: 10 50 887.52 END. 5RUNTHROUGH 14 out of 24: 12 50 1015.4 END. 5RUNTHROUGH 15 out of 24: 14 50 2000.76 END. 5RUNTHROUGH 16 out of 24: 16 50 2244.52 END. 5RUNTHROUGH 17 out of 24: 18 50 2301.52 END. 5RUNTHROUGH 18 out of 24: 20 50 2525.72 END. 5RUNTHROUGH 19 out of 24: 10 60 1292.6 END. 5RUNTHROUGH 20 out of 24: 12 60 1752.84 END. 5RUNTHROUGH 21 out of 24: 14 60 2307.2 END. 5RUNTHROUGH 22 out of 24: 16 60 2597.84 END. 5RUNTHROUGH 23 out of 24: 18 60

"Durations.csv"

UndefVarError: UndefVarError: GLMM2 not defined

In [20]:
ConditionOfInterest = [0;1]
StandardValues = [5;8]
reps = 120
PSE_Difference = 0.1
JND_Difference = 0.2
Multiplicator_PSE_Standard = 0
Multiplicator_SD_Standard = 0.108
SD_ResponseFunction = 0.1
Mean_Variability_Between = 0.1
SD_Variability_Between = 0.1
nIterations = 25
n = 20

20

In [15]:
@rput n ConditionOfInterest StandardValues reps PSE_Difference JND_Difference Multiplicator_PSE_Standard Multiplicator_SD_Standard SD_ResponseFunction Mean_Variability_Between SD_Variability_Between

    R"""
    ID = paste0("s",1:n)
        Psychometric = SimulatePsychometricFunction_Staircase(ID,
            ConditionOfInterest,
            StandardValues,
            1:reps,
            PSE_Difference,
            JND_Difference,
            Multiplicator_PSE_Standard,
            Multiplicator_SD_Standard,
            SD_ResponseFunction,
            Mean_Variability_Between,
            SD_Variability_Between)

    Psychometric$StandardValues = as.character(Psychometric$StandardValues)
    
    """
    @rget Psychometric

,ID,ConditionOfInterest,StandardValues,reps,PSE_Factor_ID,SD_Factor_ID
,Categorical…,Int64,String,Int64,Float64,Float64
1,s1,0,5,1,1.09802,0.813775
2,s2,0,5,1,1.09074,0.990893
3,s3,0,5,1,0.941488,1.14738
4,s4,0,5,1,0.868745,1.10616
5,s5,0,5,1,0.897917,0.925862
6,s6,0,5,1,0.834811,1.01861
7,s7,0,5,1,0.905171,0.86613
8,s8,0,5,1,1.19658,0.942694
9,s9,0,5,1,1.00091,0.996697


In [21]:
formulaTest = @formula(Answer ~ Difference*ConditionOfInterest + 
    (Difference + ConditionOfInterest | ID) + 
    (Difference + ConditionOfInterest | StandardValues)); 
GLMMTest2 = fit!(GeneralizedLinearMixedModel(formulaTest, Psychometric, Bernoulli()),fast=true)

formulaNull = @formula(Answer ~ Difference + ConditionOfInterest + 
    (Difference + ConditionOfInterest | ID) + 
    (Difference + ConditionOfInterest | StandardValues)); 
GLMMNull = fit!(GeneralizedLinearMixedModel(formulaNull, Psychometric, Bernoulli()),fast=true)

Generalized Linear Mixed Model fit by maximum likelihood (nAGQ = 1)
  Answer ~ 1 + Difference + ConditionOfInterest + (1 + Difference + ConditionOfInterest | ID) + (1 + Difference + ConditionOfInterest | StandardValues)
  Distribution: Bernoulli{Float64}
  Link: LogitLink()

  Deviance: 7644.8149

Variance components:
                      Column          Variance     Std.Dev.    Corr.
ID             (Intercept)          2.17325273155 1.474195622
               Difference           0.04149622707 0.203706227 -0.29
               ConditionOfInterest  0.01466494283 0.121098897 +0.55 +0.63
StandardValues (Intercept)          0.00027033388 0.016441833
               Difference           0.32848553086 0.573136573 +1.00
               ConditionOfInterest  0.00176059446 0.041959438 +1.00 +1.00

 Number of obs: 8770; levels of grouping factors: 20, 2

Fixed-effects parameters:
──────────────────────────────────────────────────────────
                     Estimate  Std.Error  z value  P(>|z|)
─

Model Formulae
1: Answer ~ 1 + Difference + ConditionOfInterest + (1 + Difference + ConditionOfInterest | ID) + (1 + Difference + ConditionOfInterest | StandardValues)
2: Answer ~ 1 + Difference + ConditionOfInterest + Difference & ConditionOfInterest + (1 + Difference + ConditionOfInterest | ID) + (1 + Difference + ConditionOfInterest | StandardValues)
──────────────────────────────────────────────────
     model-dof   deviance       χ²  χ²-dof  P(>χ²)
──────────────────────────────────────────────────
[1]         15  7698.2816                         
[2]         16  7678.5958  19.6858       1   <1e-5
──────────────────────────────────────────────────

In [22]:
(coeftable(GLMMTest)).cols[4][3]

0.7269636692649082

In [23]:
(coeftable(GLMMTest2)).cols[4][3]

0.7269636692649082

In [24]:
GLMMTest2

Generalized Linear Mixed Model fit by maximum likelihood (nAGQ = 1)
  Answer ~ 1 + Difference + ConditionOfInterest + Difference & ConditionOfInterest + (1 + Difference + ConditionOfInterest | ID) + (1 + Difference + ConditionOfInterest | StandardValues)
  Distribution: Bernoulli{Float64}
  Link: LogitLink()

  Deviance: 7630.4041

Variance components:
                      Column         Variance   Std.Dev.    Corr.
ID             (Intercept)          2.610444196 1.61568691
               Difference           0.041237879 0.20307112 -0.27
               ConditionOfInterest  0.047188960 0.21723020 -0.89 +0.67
StandardValues (Intercept)          0.000000000 0.00000000
               Difference           0.325018462 0.57010390   NaN
               ConditionOfInterest  0.004190593 0.06473479   NaN +1.00

 Number of obs: 8770; levels of grouping factors: 20, 2

Fixed-effects parameters:
─────────────────────────────────────────────────────────────────────────
                               

0.7269636692649082